## 安裝套件
! pip install transformers

! pip install datasets

! pip install torcheval

! pip install pytorch-ignite

 - transformers (4.37.0) huggingface讀取模型的套件
 - datasets (2.16.1) huggingface讀取資料集的套件
 - torcheval (0.0.7) 各種評價標準

In [2]:
import transformers as T
from datasets import load_dataset
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm import tqdm
from ignite.metrics import Rouge
import re
import pandas as pd

device = "cuda:2" if torch.cuda.is_available() else "cpu"

## 載入模型
 - 使用huggingface裝載模型的架構、參數和tokenizer
 - 保存在路徑./cache/中
 - 用.to(device)把模型裝載入訓練設備(GPU)

In [3]:
t5_model = T.T5ForConditionalGeneration.from_pretrained("google/flan-t5-base", cache_dir="./cache/").to(device)

In [11]:
# t5_tokenizer = T.T5Tokenizer.from_pretrained("google/flan-t5-base", cache_dir="./cache/") 他看不懂中文QQ
tokenizer = T.BertTokenizer.from_pretrained("google-bert/bert-base-chinese", cache_dir="./cache/", skip_special_tokens=True)

tokenizer.add_special_tokens({"pad_token": '[PAD]',
                              "cls_token": '[CLS]',
                              "sep_token": '[SEP]'
                             })

t5_model.resize_token_embeddings(len(tokenizer))

Embedding(21128, 768)

## 資料處理
使用 torch.utils.data 中的 Dataset 和 Dataloader 成批次地讀取和預處理資料

In [12]:
def get_tensor(sample):
    # 將模型的輸入和ground truth打包成Tensor
    model_inputs = tokenizer.batch_encode_plus([each[0] for each in sample], padding=True, truncation=True, return_tensors="pt")
    model_outputs = tokenizer.batch_encode_plus([each[1] for each in sample], padding=True, truncation=True, return_tensors="pt")
    return model_inputs["input_ids"].to(device), model_outputs["input_ids"].to(device)

class LCSTSDataset(Dataset):
    def __init__(self, split="train") -> None:
        super().__init__()
        assert split in ["train[:20000]", "validation[:10000]", "test"]
        data_df = load_dataset("hugcyp/LCSTS", split=split, cache_dir="./cache/").to_pandas()
        data_df['text'] = data_df['text'].apply(lambda text: "文本摘要：" + text)
        self.data = data_df[['text', 'summary']].values.tolist()
        
    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)

In [13]:
data_sample = LCSTSDataset(split="train[:20000]").data[:3]

print(f"Dataset example: \n{data_sample[0]} \n{data_sample[1]}")

Dataset example: 
['文本摘要：新华社受权于18日全文播发修改后的《中华人民共和国立法法》，修改后的立法法分为“总则”“法律”“行政法规”“地方性法规、自治条例和单行条例、规章”“适用与备案审查”“附则”等6章，共计105条。', '修改后的立法法全文公布'] 
['文本摘要：一辆小轿车，一名女司机，竟造成9死24伤。日前，深圳市交警局对事故进行通报：从目前证据看，事故系司机超速行驶且操作不当导致。目前24名伤员已有6名治愈出院，其余正接受治疗，预计事故赔偿费或超一千万元。', '深圳机场9死24伤续：司机全责赔偿或超千万']


## 超參數
 - 學習率 (learning rate): 1e-5
 - 訓練輪數 (epochs): 3
 - 優化器 (optimizer): AdamW
 - 批次大小 (batch size): 8
 - 評量指標 (evaluation matrics)Rouge-2

In [39]:
lr = 1e-5
epochs = 3
optimizer = AdamW(t5_model.parameters(), lr = 1e-5)

train_batch_size = 8
validation_batch_size = 8

rouge = Rouge(variants=["L", 2], multiref="best")

output_max_length = 200

In [22]:
LCSTS_train = DataLoader(LCSTSDataset(split="train[:20000]"),
                         collate_fn=get_tensor,
                         batch_size=train_batch_size,
                         shuffle=True)

LCSTS_validation = DataLoader(LCSTSDataset(split="validation[:10000]"),
                              collate_fn=get_tensor,
                              batch_size=validation_batch_size,
                              shuffle=False)

## 驗證
驗證程式
 - 將驗證資料輸入模型，用Rouge-2評價輸出的效果
 - Rouge的使用方法參考 https://pytorch.org/ignite/generated/ignite.metrics.Rouge.html

In [28]:
def evaluate(model):
    pbar = tqdm(LCSTS_validation)
    pbar.set_description(f"Evaluating")

    for inputs, summary in pbar:
        output = [tokenizer.batch_decode(model.generate(inputs, max_length=output_max_length))]
        summary = [tokenizer.batch_decode(summary)]
        for i in range(len(output)):
            for s in output:
                rouge.update(([s], [summary]))
    return rouge.compute()

## T5 訓練 & 測試
 - 將資料成批次輸入T5模型，並獲取其損失函數數值，隨後計算梯度優化
 - tqdm用來顯示模型的訓練進度

In [24]:
for ep in range(epochs):
    pbar = tqdm(LCSTS_train)
    pbar.set_description(f"Training epoch [{ep+1}/{epochs}]")
    for inputs, targets in pbar:
        optimizer.zero_grad()
        loss = t5_model(input_ids=inputs, labels=targets).loss
        loss.backward()
        optimizer.step()
        pbar.set_postfix(loss = loss.item())
    # torch.save(t5_model, f'./saved_models/ep{ep}.mod')

Training epoch [3/3]: 100%|███████████████████████████████████████████████████████| 2500/2500 [08:32<00:00,  4.87it/s, loss=3.48]


In [29]:
def simplify(s):
    return s.replace('[CLS]','').replace('[PAD]','').replace('[SEP]','').replace('[UNK]','')

In [30]:
inputs, summary = next(iter(LCSTS_validation))
x = [tokenizer.batch_decode(inputs)]
y = [tokenizer.batch_decode(summary)]
pred_y = [tokenizer.batch_decode(t5_model.generate(inputs, max_length=output_max_length))]
    
for i in range(len(x)):
    for j in range(len(x[0])):
        print("input: ", simplify(x[i][j]))
        print("output: ", simplify(pred_y[i][j]))
        print("answer: ", simplify(y[i][j]))
        print("")

input:   文 本 摘 要 ： 日 前 ， 方 舟 子 发 文 直 指 林 志 颖 旗 下 爱 碧 丽 推 销 假 保 健 品 ， 引 起 哗 然 。 调 查 发 现 ， 爱 碧 丽 没 有 自 己 的 生 产 加 工 厂 。 其 胶 原 蛋 白 饮 品 无 核 心 研 发 ， 全 部 代 工 生 产 。 号 称 有  逆 生 长  功 效 的 爱 碧 丽  梦 幻 奇 迹 限 量 组  售 价 高 达 1080 元 ， 实 际 成 本 仅 为 每 瓶 4 元 ！      
output:    方 舟 子 发 文 直 指 林 志 颖 旗 下 爱 碧                                                
answer:   林 志 颖 公 司 疑 涉 虚 假 营 销 无 厂 房 无 研 发      

input:   文 本 摘 要 ： 韩 方 应 对 路 径 可 以 概 括 为 ： 企 业 道 歉 担 责 ； 政 府 公 正 不 护 短 ； 民 间 祈 福 关 怀 。 他 们 深 知 形 象 的 重 要 ， 竭 力 呵 护 企 业 品 牌 和 国 家 形 象 。 正 如 有 评 论 ， 韩 国  政 府 + 企 业 + 民 众  三 位 一 体 式 呵 护 韩 国 国 家 形 象 的  苦 心 经 营  ， 的 确 有 值 得 我 们 借 鉴 之 处 。       
output:    韩 国  政 府 + 企 业 + 民 众  三 位 一 体 式 呵 护 韩 国 家 形 象                                      
answer:   从 韩 亚 航 空 事 故 看 其 应 对 路 径          

input:   文 本 摘 要 ： 63 岁 退 休 教 师 谢 淑 华 ， 拉 着 人 力 板 车 ， 历 时 1 年 ， 走 了 2 万 4 千 里 路 ， 带 着 年 过 九 旬 的 妈 妈 环 游 中 国 ， 完 成 了 妈 妈  一 辈 子 在 锅 台 边 转 ， 也 想 出 去 走 走  的 心 愿 。 她 说 ：  妈 妈 愿 意 出 去 走 走 ， 我 就 愿 意 拉 着 ， 孝 心 不 能 等 ， 能 走 多 远 就 走 多 远 。        

In [31]:
print(f"Rouge-2 score on epoch {ep}:", evaluate(t5_model))

Evaluating: 100%|████████████████████████████████████████████████████████████████████████████| 1086/1086 [24:10<00:00,  1.34s/it]

Rouge-2 score on epoch 2: {'Rouge-L-P': 0.0, 'Rouge-L-R': 0.0, 'Rouge-L-F': 0.0, 'Rouge-2-P': 0.0, 'Rouge-2-R': 0.0, 'Rouge-2-F': 0.0}


## GPT2 訓練 & 測試
 - 將資料成批次輸入模型，並獲取其損失函數數值，隨後計算梯度優化
 - tqdm用來顯示模型的訓練進度

In [32]:
def get_GPT2_tensor(sample):
    encoded_x = tokenizer.batch_encode_plus([each[0] for each in sample], padding=True, truncation=True, return_tensors="pt")
    encoded_y = tokenizer.batch_encode_plus([each[1] for each in sample], padding=True, truncation=True, return_tensors="pt")
    return encoded_x["input_ids"].to(device), encoded_x["attention_mask"].to(device), encoded_y["input_ids"].to(device)

In [33]:
LCSTS_GPT2_train = DataLoader(LCSTSDataset(split="train[:20000]"),
                              collate_fn=get_GPT2_tensor,
                              batch_size=train_batch_size,
                              shuffle=True)

LCSTS_GPT2_validation = DataLoader(LCSTSDataset(split="validation[:10000]"),
                                   collate_fn=get_GPT2_tensor,
                                   batch_size=validation_batch_size,
                                   shuffle=False)

In [34]:
GPT2_model = T.GPT2LMHeadModel.from_pretrained("openai-community/gpt2", cache_dir="./cache/").to(device)
tokenizer = T.BertTokenizer.from_pretrained("google-bert/bert-base-chinese", cache_dir="./cache/")
optimizer = AdamW(GPT2_model.parameters(), lr = 1e-5)

In [35]:
GPT2_model.resize_token_embeddings(len(tokenizer))

Embedding(21128, 768)

In [36]:
GPT2_model = GPT2_model.to(device)
GPT2_model.train()

for ep in range(epochs):
    pbar = tqdm(LCSTS_GPT2_train)
    pbar.set_description(f"Training epoch [{ep+1}/{epochs}]")
    for input_ids, attention_mask, summary in pbar:
        optimizer.zero_grad()
        outputs = GPT2_model(input_ids=input_ids, labels=input_ids, attention_mask=attention_mask)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        pbar.set_postfix(loss = loss.item())
    # torch.save(t5_model, f'./saved_models/ep{ep}.mod')

Training epoch [3/3]: 100%|███████████████████████████████████████████████████████| 2500/2500 [05:51<00:00,  7.12it/s, loss=5.77]


In [49]:
inputs, attention_mask, summary = next(iter(LCSTS_GPT2_validation))
x = [tokenizer.batch_decode(inputs)]
y = [tokenizer.batch_decode(summary)]
pred_y = [tokenizer.batch_decode(GPT2_model.generate(inputs,
                                                        max_length=output_max_length, 
                                                        attention_mask=attention_mask,
                                                        pad_token_id=50256)# hugging face produce error if pad_token_id is not set
                                        )]
    
for i in range(len(x)):
    for j in range(len(x[0])):
        print("input: ", simplify(x[i][j]))
        print("output: ", simplify(pred_y[i][j]))
        print("answer: ", simplify(y[i][j]))
        print("")

input:   文 本 摘 要 ： 日 前 ， 方 舟 子 发 文 直 指 林 志 颖 旗 下 爱 碧 丽 推 销 假 保 健 品 ， 引 起 哗 然 。 调 查 发 现 ， 爱 碧 丽 没 有 自 己 的 生 产 加 工 厂 。 其 胶 原 蛋 白 饮 品 无 核 心 研 发 ， 全 部 代 工 生 产 。 号 称 有  逆 生 长  功 效 的 爱 碧 丽  梦 幻 奇 迹 限 量 组  售 价 高 达 1080 元 ， 实 际 成 本 仅 为 每 瓶 4 元 ！      
output:   文 本 摘 要 ： 日 前 ， 方 舟 子 发 文 直 指 林 志 颖 旗 下 爱 碧 丽 推 销 假 保 健 品 ， 引 起 哗 然 。 调 查 发 现 ， 爱 碧 丽 没 有 自 己 的 生 产 加 工 厂 。 其 胶 原 蛋 白 饮 品 无 核 心 研 发 ， 全 部 代 工 生 产 。 号 称 有  逆 生 长  功 效 的 爱 碧 丽  梦 幻 奇 迹 限 量 组  售 价 高 达 1080 元 ， 实 际 成 本 仅 为 每 瓶 4 元 ！                                                                                    
answer:   林 志 颖 公 司 疑 涉 虚 假 营 销 无 厂 房 无 研 发      

input:   文 本 摘 要 ： 韩 方 应 对 路 径 可 以 概 括 为 ： 企 业 道 歉 担 责 ； 政 府 公 正 不 护 短 ； 民 间 祈 福 关 怀 。 他 们 深 知 形 象 的 重 要 ， 竭 力 呵 护 企 业 品 牌 和 国 家 形 象 。 正 如 有 评 论 ， 韩 国  政 府 + 企 业 + 民 众  三 位 一 体 式 呵 护 韩 国 国 家 形 象 的  苦 心 经 营  ， 的 确 有 值 得 我 们 借 鉴 之 处 。       
output:   文 本 摘 要 ： 韩 方 应 对 路 径 可 以 概 括 为 ： 企 业 道 歉 担 责 ； 政 府 公 正 不 护 短 ； 民 间 祈 福 关 怀 。 他 们 深 知 形 象 的 重 要 ， 竭 力 呵 护 企 业 品 牌 和 国 家 形 象 。 正 如 有 

In [50]:
def evaluate(model):
    pbar = tqdm(LCSTS_GPT2_validation)
    pbar.set_description(f"Evaluating")

    for inputs, attention_mask, summary in pbar:
        # model output
        output = [tokenizer.batch_decode(model.generate(inputs,
                                                        max_length=output_max_length, 
                                                        attention_mask=attention_mask,
                                                        pad_token_id=50256)# hugging face produce error if pad_token_id is not set
                                        )]
        # 正確答案
        summary = [tokenizer.batch_decode(summary)]
        for i in range(len(output)):
            for s in output:
                rouge.update(([s], [summary]))
    return rouge.compute()

In [51]:
print(f"Rouge-2 score on epoch {ep}:", evaluate(GPT2_model))

Evaluating: 100%|████████████████████████████████████████████████████████████████████████████| 1086/1086 [20:29<00:00,  1.13s/it]

Rouge-2 score on epoch 2: {'Rouge-L-P': 0.0, 'Rouge-L-R': 0.0, 'Rouge-L-F': 0.0, 'Rouge-2-P': 0.0, 'Rouge-2-R': 0.0, 'Rouge-2-F': 0.0}
